In [1]:
!pip install language-tool-python

In [2]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.0 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer, util
import language_tool_python
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import numpy as np
import string

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
# Initialize the grammar checker
grammar_tool = language_tool_python.LanguageTool('en-US')
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

INFO:language_tool_python.download_lt:Unzipping /tmp/tmpcmnsjbfc.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:


def keyword_matching(key_answer, student_answer):
    key_tokens = word_tokenize(key_answer.lower())
    student_tokens = word_tokenize(student_answer.lower())
    key_tokens = [word for word in key_tokens if word not in stopwords.words('english') and word not in string.punctuation]
    student_tokens = [word for word in student_tokens if word not in stopwords.words('english') and word not in string.punctuation]

    matches = set(key_tokens) & set(student_tokens)
    score = (len(matches) / len(key_tokens) * 10) if key_tokens else 0
    return score

def synonym_check(key_answer, student_answer):
    key_tokens = word_tokenize(key_answer.lower())
    student_tokens = word_tokenize(student_answer.lower())

    synonyms_found = 0
    for word in student_tokens:
        for syn in wordnet.synsets(word):
            if syn.name().split('.')[0] in key_tokens:
                synonyms_found += 1
                break

    score = (synonyms_found / len(key_tokens) * 10) if key_tokens else 0
    return score

def grammar_check(student_answer):
    matches = grammar_tool.check(student_answer)
    num_errors = len(matches)
    if num_errors == 0:
        return 10
    elif num_errors < 3:
        return 8
    elif num_errors < 6:
        return 6
    else:
        return 4

def contextual_analysis(key_answer, student_answer):
    key_embedding = model.encode(key_answer, convert_to_tensor=True)
    student_embedding = model.encode(student_answer, convert_to_tensor=True)

    # Calculate cosine similarity
    cosine_similarity = util.pytorch_cos_sim(key_embedding, student_embedding)
    score = cosine_similarity.item() * 10  # Scale the score to 0-10
    return score

def evaluate_answer(key_answer, student_answer):
    # Calculate individual scores
    keyword_score = keyword_matching(key_answer, student_answer)
    synonym_score = synonym_check(key_answer, student_answer)
    grammar_score = grammar_check(student_answer)
    context_score = contextual_analysis(key_answer, student_answer)

    # Adjusted weights
    final_score = (keyword_score * 0.3) + (synonym_score * 0.15) + (grammar_score * 0.10) + (context_score * 0.45)
    return final_score

In [7]:
# Example usage
key_answer = (
    "The water was evaporated, leaving the salt."
)

student_answer = (
    "By letting it sit in a dish for a day."
)

score = evaluate_answer(key_answer, student_answer)
print(f"Score: {score:.2f}/10")

Score: 2.00/10


In [8]:
# Example usage
key_answer = (
    "The water was evaporated, leaving the salt."
)

student_answer = (
    "Let the water evaporate and the salt is left behind."
)

score = evaluate_answer(key_answer, student_answer)
print(f"Score: {score:.2f}/10")

Score: 6.46/10


In [9]:
# Example usage
key_answer = (
    "By initializing a pointer to point to the first element of the array, and then incrementing this pointer with the index of the array element."
)

student_answer = (
    "multi-dimensional array"
)

score = evaluate_answer(key_answer, student_answer)
print(f"Score: {score:.2f}/10")

Score: 2.84/10


In [10]:
# Example usage
key_answer = (
    "To simulate the behaviour of portions of the desired software product."
)

student_answer = (
    "To address major issues in the creation of the program. There is no way to account for all possible bugs in the program, but it is possible to prove the program is tangible."
)

score = evaluate_answer(key_answer, student_answer)
print(f"Score: {score:.2f}/10")

Score: 3.30/10
